# UV-Vis Data Collection Project

In [2]:
import pandas as pd

Below a table will be constructed containing the UV-vis data from a crystal violet experiment. There will be five reactions with varying concentrations of NaOH in M. The slope represents the absorbance over time, as the violet color from crystal violet fades over time as the reaction proceeds.

In [3]:
uvvis = pd.read_csv("crystalviolet3", delim_whitespace=False,
                    index_col=0, parse_dates=True)
print(uvvis)

          Concentration     Slope     Slope.Err
Reaction                                       
1                  0.10 -0.000292  7.103950e-06
2                  0.08 -0.000262  5.629424e-06
3                  0.06 -0.000266  4.247431e-06
4                  0.04 -0.000190  2.107726e-06
5                  0.02 -0.000088  5.449833e-07


C:\Users\cson4\AppData\Local\Temp\ipykernel_12388\1257987019.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  uvvis = pd.read_csv("crystalviolet3", delim_whitespace=False,
C:\Users\cson4\AppData\Local\Temp\ipykernel_12388\1257987019.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  uvvis = pd.read_csv("crystalviolet3", delim_whitespace=False,


## Curve Fitting 

In [2]:
import numpy as numpy

This imports a collection of functions know as numpy, and I named it as numpy as well to keep track of it.

In [4]:
from scipy.optimize import curve_fit

This imports the curve fit function from the SciPy reference manual. This specific function was found under the scipy.optimize section which includes methods of solving nonlinear data sets, though I am also using it for a linear set for the sake of consistency. 

In [6]:
def first(x, a, b):
    return a * np.exp(-b * x)

This code defines a function that I have titled "first" and it will be used to fit the data to a first-order, exponential line. The parameter "a" is the initial concentration, "b" is the rate constant k, and "x" is the time.

In [9]:
def zero(x, a, b):
    return -a * x + b

This code defines a function that I have titled "zero" and it will be used to fit the data to a zero-order, linear line. The parameter "a" is the rate constant k, "b" is the initial concentration, and "x" is the time.

In [8]:
def second(x, a, b):
    return (a) / (a * b * x + 1)

This code defines a function that I have titled "second" and it will be used to fit the data to a second-order line. The parameter "a" is the initial concentration, "b" is the rate constant k, and "x" is time.

FINDING THE AIC VALUE